In [1]:
import h5py
import pandas as pd
from policyengine_uk import Microsimulation

weight_data = h5py.File("constituencies/weights.h5", "r")
constituencies = pd.read_csv("constituencies/targets/constituencies.csv")

In [2]:
baseline = Microsimulation()

hnet_24 = baseline.calculate("real_household_net_income", 2024)
hnet_29 = baseline.calculate("real_household_net_income", 2029)

In [3]:
change = hnet_29 - hnet_24

In [4]:
from constituencies.charts import constituencies as hex_con

In [15]:
import plotly.express as px
import numpy as np

weights = weight_data["weight"]


weighted_gain = np.dot(weights, change)
weighted_original_income = np.dot(weights, hnet_24)
weighted_rel_gain = weighted_gain / weighted_original_income

df = pd.DataFrame()
df["Average real change (%)"] = weighted_rel_gain * 100
df["Constituency"] = constituencies.name
df["Code"] = constituencies.code

df["y"] = hex_con.set_index("code").loc[df.Code].y.values
df["x"] = hex_con.set_index("code").loc[df.Code].x.values
df.x = df.x + (df.y % 2 == 0) * -0.5

In [16]:
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="Average real change (%)",
    hover_data=["Constituency"],
)

from policyengine_core.charts import *

fig.update_layout(
    height=580,
    width=700,
)
# Set zero to white
# fig.update_coloraxes(cmax=-df["Average real change"].min(), cmin=df["Average real change"].min())

# Set marker to hexagon
fig.update_traces(marker=dict(symbol="hexagon", size=12))

fig.update_layout(
    title="Baseline forecast for real household net income growth (2024-29)",
    xaxis_tickvals=[],
    xaxis_title="",
    yaxis_tickvals=[],
    yaxis_title="",
    # No bg
    plot_bgcolor="rgba(0,0,0,0)",
    legend_title="Average household net income change",
    # Set color axis to %
    title_font_color="black",
    legend_title_font_color="black",
)

In [7]:
df[["Code", "Constituency", "Average real change (%)"]].to_csv(
    "constituency_baseline_forecast.csv", index=False
)

In [8]:
df["Average real change (%)"].describe()

count    650.000000
mean       1.737540
std        1.905879
min       -5.249496
25%        0.602416
50%        1.596063
75%        2.644899
max       10.635343
Name: Average real change (%), dtype: float64

In [9]:
df.sort_values("Average real change (%)")

,Average real change (%),Constituency,Code,y,x
565,-5.249496,Rhondda,W07000052,11,10.0
572,-3.151484,Ynys Môn,W07000041,19,7.0
36,-2.936147,Blackpool South,E14000573,26,12.5
241,-2.801534,Stoke-on-Trent Central,E14000972,23,17.0
242,-2.744494,Stoke-on-Trent North,E14000973,24,16.5
...,...,...,...,...,...
276,9.448778,Epping Forest,E14000693,13,22.0
457,9.456671,Runnymede and Weybridge,E14000907,8,17.5
90,9.520226,Tatton,E14000987,17,13.0
352,10.480521,Finchley and Golders Green,E14000703,11,22.0
